工程化安全写法（强烈推荐）
(priority, seq, data)

- priority：优先级
- seq：自增序号（保证稳定性）
- data：真实任务


👉 同优先级，按进入顺序稳定处理

In [2]:
from queue import PriorityQueue
from itertools import count

counter = count()
q = PriorityQueue(maxsize=3)

q.put((1, next(counter), "task a"))
q.put((0, next(counter), "task b"))
q.put((1, next(counter), "task c"))


while not q.empty():
    _, _, task = q.get()
    print(f"消费任务{task}")


消费任务task b
消费任务task a
消费任务task c


In [ ]:
from queue import PriorityQueue
from itertools import count
import threading

Task = tuple[int, int, str]


def consumer(
    q: PriorityQueue[tuple[int, int, str]],
    slots: list[str],
    out_index: list[int],
    out_lock: threading.Lock,
    stop_value: str = "__STOP__",
):
    while True:
        got_item = False
        task: tuple[int, int, str] | None = None

        try:
            task = q.get()
            got_item = True

            priority, seq, value = task
            if value == stop_value:
                print("【消费者】消费任务完成")
                return
            # 分析：
            # ✅ PriorityQueue 已经保证取出顺序：priority 小的先，priority 相同按 seq
            #    所以我们按“取出次序”写入 slots[0..n-1]
            with out_lock:
                idx = out_index[0]
                out_index[0] += 1
            slots[idx] = f"done: {value}"
            print(f"【消费者】消费任务：{task}")
        finally:
            if got_item:
                q.task_done()


def run_priority_tasks(
    tasks: list[tuple[int, str]], workers: int = 2
) -> list[str | None]:
    priority_queue: PriorityQueue[Task] = PriorityQueue()
    n = len(tasks)
    results: list[str | None] = [None] * n
    out_index = [0]
    out_lock = threading.Lock()
    # ✅ counter 放在函数内部，每次从 0 开始
    counter = count()

    threads = [
        threading.Thread(
            target=consumer,
            args=(priority_queue, results, out_index, out_lock),
        )
        for _ in range(workers)
    ]

    for c in threads:
        c.start()

    for _, task in enumerate(tasks):
        priority_queue.put(
            (
                task[0],
                next(counter),
                task[1],
            )
        )

    for index, c in enumerate(threads):
        priority_queue.put((10**9, next(counter), "__STOP__"))

    priority_queue.join()
    for c in threads:
        c.join()

    # 检查：如果任务中出现None则抛出异常
    if any(item is None for item in results):
        raise ValueError(f"丢失任务：{results}")

    print(f"out_index: {out_index}")
    return [item for item in results if item is not None]


tasks = [
    (2, "low"),
    (0, "urgent"),
    (1, "normal"),
    (1, "normal-2"),
]
assert run_priority_tasks([]) == []
assert run_priority_tasks(tasks) == [
    "done: urgent",
    "done: normal",
    "done: normal-2",
    "done: low",
]


【消费者】消费任务完成【消费者】消费任务完成

out_index: [0]
【消费者】消费任务：(0, 1, 'urgent')
【消费者】消费任务：(1, 2, 'normal')
【消费者】消费任务：(1, 3, 'normal-2')
【消费者】消费任务：(2, 0, 'low')
【消费者】消费任务完成
【消费者】消费任务完成
out_index: [4]


In [ ]:
from queue import PriorityQueue, Empty
from itertools import count
import threading
import time
import random


"""
多 worker 消费 PriorityQueue
用 stop_event 控制退出
退出前仍然把已取到的任务处理完
主线程可随时 stop_event.set() 触发停止
"""


Task = tuple[int, int, str]  # (priority, seq, value)


def worker(
    q: PriorityQueue[Task],
    slots: list[str | None],
    out_index: list[int],
    out_lock: threading.Lock,
    stop_event: threading.Event,
) -> None:
    while True:
        # ✅ 停止条件：红灯亮 + 队列已空 -> 退出
        if stop_event.is_set() and q.empty():
            return

        try:
            task = q.get(timeout=0.2)  # ✅ 不永久阻塞
        except Empty:
            continue

        try:
            priority, seq, value = task
            # 模拟处理耗时
            time.sleep(random.uniform(0.1, 0.3))

            with out_lock:
                idx = out_index[0]
                out_index[0] += 1

            slots[idx] = f"done:{value}"
        finally:
            q.task_done()


def run_priority_tasks_event(
    tasks: list[tuple[int, str]],
    workers: int = 2,
    maxsize: int = 5,
    cancel_after: float | None = None,  # 传秒数就演示“中途取消”
):
    q: PriorityQueue[Task] = PriorityQueue(maxsize=maxsize)
    n = len(tasks)
    slots: list[str | None] = [None] * n

    out_index = [0]
    out_lock = threading.Lock()
    stop_event = threading.Event()
    counter = count()

    # 创建线程
    threads = [
        threading.Thread(
            target=worker,
            args=(q, slots, out_index, out_lock, stop_event),
            name=f"worker-{i}",
        )
        for i in range(workers)
    ]

    # 开启消费者
    for t in threads:
        t.start()

    # 生产任务
    for priority, value in tasks:
        q.put((priority, next(counter), value))

    # 是否取消
    if cancel_after is not None:
        print(f"取消 {cancel_after} 秒后取消任务")
        stop_event.set()
    q.join()
    for t in threads:
        t.join()
    # 如果取消了，可能 slots 后半段仍是 None，这里只返回已完成的
    done = [item for item in slots if item is not None]
    return done


tasks = [
    (2, "low"),
    (0, "urgent"),
    (1, "normal"),
    (1, "normal-2"),
]

out = run_priority_tasks_event(tasks, workers=2)
# 注意：这个版本“停止信号”不保证一定处理完所有任务（取决于你何时 set）
# 这里我们是生产完再 set，所以会处理完
assert out == ["done:urgent", "done:normal", "done:normal-2", "done:low"]
print("OK", out)
